In [1]:
import time
import sys
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bt
import h5py

from datetime import datetime
import yfinance as yf

In [2]:
tickers_new = 'aapl,msft,c,gs,ge,jnj,pg,ko,amzn,jpm,adbe,ma,dis,txn'
data = bt.get(
    tickers_new,
    start='2010-01-01',
    end = '2022-07-01'
)
data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,aapl,msft,c,gs,ge,jnj,pg,ko,amzn,jpm,adbe,ma,dis,txn
Date,,,,,,,,,,,,,,
2010-01-04,6.479000,23.474920,26.893303,136.536972,67.992630,42.974476,40.160065,18.498014,6.695000,29.669838,37.090000,23.853483,27.843176,18.196344
2010-01-05,6.490200,23.482504,27.921570,138.950897,68.344727,42.476151,40.173210,18.274250,6.734500,30.244555,37.700001,23.782892,27.773718,18.091410
2010-01-06,6.386965,23.338390,28.791641,137.467789,67.992630,42.821640,39.982658,18.267767,6.612500,30.410713,37.619999,23.748520,27.626127,17.958481
2010-01-07,6.375156,23.095688,28.870752,140.157822,71.513329,42.516014,39.765812,18.222359,6.500000,31.013124,36.889999,23.593330,27.634809,18.014456
2010-01-08,6.417542,23.254957,28.396160,137.507263,73.053604,42.662189,39.713253,17.885090,6.676000,30.936968,36.689999,23.601690,27.678221,18.427214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,140.444199,264.041077,44.782074,289.352936,52.030647,174.613449,138.010971,60.242382,116.459999,111.237900,387.720001,326.949341,97.462357,149.027634
2022-06-27,140.444199,261.269470,44.754002,287.470154,51.805439,174.450592,136.959244,60.118149,113.220001,110.346619,381.070007,325.297272,96.296165,148.712280
2022-06-28,136.260437,252.974411,44.173870,286.237244,51.160873,169.488739,134.731567,59.516106,107.400002,109.815666,365.630005,314.870514,95.608398,147.317078


In [4]:
sma50 = data.rolling(50).mean()
sma200 = data.rolling(200).mean()

In [46]:
class WeighTarget(bt.Algo):
    def __init__(self, target_weights):
        self.tw = target_weights

    def __call__(self, target):
        if target.now in self.tw.index:
            w = self.tw.loc[target.now]
            target.temp['weights'] = w.dropna()

        return True
    
st = time.time()

## now we need to calculate our target weight DataFrame
# first we will copy the sma200 DataFrame since our weights will have the same strucutre
tw = sma200.copy()

# set appropriate target weights
tw[sma50 > sma200] = 1.0
tw[sma50 <= sma200] = -1.0

# divide by row sum to normalize
tw = tw.div(tw.abs().sum(axis=1), axis=0)

# replace nans with 0.0
tw[sma200.isnull()] = 0.0

ma_cross = bt.Strategy('ma_cross', [WeighTarget(tw),
                                    bt.algos.Rebalance()])

t = bt.Backtest(
    ma_cross,
    data,
    initial_capital=50000000.0,
    commissions=lambda q, p: abs(q*p)*.0001,
)
res = bt.run(t)
et = time.time()
print(f"Run time: {et-st:.2f} seconds")
res.display()

Run time: 1.98 seconds
Stat                 ma_cross
-------------------  ----------
Start                2010-01-03
End                  2022-06-30
Risk-free rate       0.00%

Total Return         130.93%
Daily Sharpe         0.57
Daily Sortino        0.86
CAGR                 6.93%
Max Drawdown         -31.52%
Calmar Ratio         0.22

MTD                  7.51%
3m                   13.87%
6m                   9.11%
YTD                  9.19%
1Y                   17.19%
3Y (ann.)            6.48%
5Y (ann.)            7.98%
10Y (ann.)           9.06%
Since Incep. (ann.)  6.93%

Daily Sharpe         0.57
Daily Sortino        0.86
Daily Mean (ann.)    7.59%
Daily Vol (ann.)     13.23%
Daily Skew           -0.79
Daily Kurt           25.29
Best Day             10.30%
Worst Day            -10.28%

Monthly Sharpe       0.65
Monthly Sortino      1.04
Monthly Mean (ann.)  7.42%
Monthly Vol (ann.)   11.44%
Monthly Skew         -0.74
Monthly Kurt         1.75
Best Month           9.19%
Worst M